## Tooling - GSW standalone board

In [1]:
import Switch as sw

module ser02.py: list_ports(), open_port("COMx"), r(addr), w(addr, value), close_ports()
device: /dev/ttyS0
name: ttyS0
description: n/a
device_path: /sys/devices/platform/soc/fe215040.serial
hwid: n/a
subsystem: platform

device: /dev/ttyUSB3
name: ttyUSB3
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.3/ttyUSB3
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.3
subsystem: usb-serial

device: /dev/ttyUSB2
name: ttyUSB2
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.2/ttyUSB2
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.2
subsystem: usb-serial

device: /dev/ttyUSB1
name: ttyUSB1
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.1/ttyUSB1
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.1
subsystem: usb-serial

device: /dev

In [3]:
import sys
print (224*0x1000000)
print ("%#x"%(224*0x1000000))
print ("%#x"%sys.maxsize)

3758096384
0xe0000000
0x7fffffff


#### Check if microcode enabled
After microcode download, reg0x456 = 0x0008 (**the parser microcode is valid**), reg0x457 = 0x0201 (**bit1=0 PCE is enabled**).

Without microcode downloaded, reg0x456 = 0x0000, reg0x457 = 0x0001

0x456 bit 3
<img src="GSW_reg0x456_bit3.PNG">

In [2]:
print ("0x%04x"%sw.r(0xf383))
print ("0x%04x"%sw.r(0xf384))
print ("0x%04x"%sw.r(0x456))
print ("0x%04x"%sw.r(0x457))

0x03c3
0x003f
0x0000
0x0001


#### Download microcode
It takes 25sec.

In [2]:
sw.pyGSW_HW_Init()
print ("finished.")

finished.


### Set port 0 egress rate shaping
Port 0 default queue 0 <- rate shaping ID 0.

- pyGSW_QoS_ShaperCfgSet:  nRateShaperId = 0, bEnable = 0, bAVB = 0, nCbs = 0, nRate = 0
- pyGSW_QoS_ShaperQueueAssign(nRateShaperId = 0, nQueueId = 0, pr = 1):

In [30]:
'''
for tc in range(16):
    sw.pyGSW_QoS_QueuePortGet(1, tc)
'''
sw.pyGSW_QoS_ShaperCfgSet(0, 1, 0, 2000, 38*1024)
sw.pyGSW_QoS_ShaperQueueAssign(0, 0)
sw.pyGSW_QoS_ShaperQueueAssign(0, 1)
sw.pyGSW_QoS_ShaperQueueAssign(0, 2)
sw.pyGSW_QoS_ShaperQueueAssign(0, 3)


{'ret': 0}

### Test PCP QoS
- pyGSW_QoS_PortCfgSet(nPortId = 0, eClassMode = 0, nTrafficClass = 0, 
- pyGSW_QoS_SchedulerCfgSet(nQueueId = 0, eType = 0, nWeight = 0,

In [29]:
#pcp_tc = [0,0,0,0,0,2,0,0]
#print (sw.pyGSW_QoS_PCP_ClassSet(7, pcp_tc))
print (sw.pyGSW_QoS_PortCfgSet(1,0,2))
print (sw.pyGSW_QoS_SchedulerCfgSet(2, 0, 0xfffe))
#print (sw.pyGSW_QoS_SchedulerCfgSet(2, 0, 0))

{'ret': 0}
{'ret': 0}


In [3]:
sw.pyGSW_QoS_SchedulerCfgGet(0)
print("")
sw.pyGSW_QoS_SchedulerCfgGet(2)

GSW_QoS_SchedulerCfgGet    ---    Return value: GSW_statusOk 
nQueueId = 0x0, 0 
eType = 0x0, 0 GSW_QOS_SCHEDULER_STRICT  [GSW_QoS_Scheduler_t]
nWeight = 0x1800, 6144 

GSW_QoS_SchedulerCfgGet    ---    Return value: GSW_statusOk 
nQueueId = 0x2, 2 
eType = 0x0, 0 GSW_QOS_SCHEDULER_STRICT  [GSW_QoS_Scheduler_t]
nWeight = 0x1800, 6144 


{'ret': 0, 'nQueueId': '2', 'eType': '0', 'nWeight': '6144'}

- pyGSW_QoS_MeterCfgSet(bEnable, nMeterId, nCbs, nEbs, nRate)
- pyGSW_QoS_MeterPortAssign(nMeterId, eDir, nPortIngressId, nPortEgressId)
- pyGSW_QoS_WredCfgSet(eProfile, eThreshMode, nRed_Min, nRed_Max, nYellow_Min, nYellow_Max, nGreen_Min, nGreen_Max)
- pyGSW_QoS_WredPortCfgSet(nPortId, nRed_Min, nRed_Max, nYellow_Min, nYellow_Max, nGreen_Min, nGreen_Max)

In [26]:
print (sw.pyGSW_QoS_MeterCfgSet(1, 4, 4000, 4000, 50*1024))
print (sw.pyGSW_QoS_MeterPortAssign(4, 2, 0, 0))

print (sw.pyGSW_QoS_WredPortCfgSet(0, 0,0,0,0,65536,65536))

#print (sw.pyGSW_QoS_MeterCfgSet(1, 0, 4000, 4000, 50*1024))
#print (sw.pyGSW_QoS_MeterPortAssign(0, 1, 0, 0))

#print( sw.pyGSW_QoS_WredCfgSet(0,0,0,0,0,0,65536,65536))

{'ret': 0}
{'ret': 0}
{'ret': 0}


#### List of capabilities

In [3]:
ret = {"nCapType": "0"}
a = 0
while ret["nCapType"]!="13":
    ret = sw.pyGSW_CapGet(a, pr=0)
    #print ( "nCapType", ret["nCapType"])
    desc = ret["cDesc[128]"][2:-1].split()
    cap = ""
    for s in desc:
        if (s == "0x00"):
            break
        else:
            cap = cap + (bytes.fromhex(s[2:]).decode('utf-8'))
    print (cap, ret["nCap"])
    a = a+1

Number of Ethernet ports 7
Number of virtual ports 6
Pcket buffer size[in Bytes]: 131072
Buffer Segment size: 256
Number of queues: 32
Number of traffic meters: 16
Number of rate shapers: 32
Number of VLAN groups: 64
Number of FIDs: 64
Number of MAC entries: 2048
Number of multicast entries 64
Number of PPPoE sessions: 16
Number of STAG VLAN groups: 64
 0


#### MAC Table read
| Device            | MAC addr          |
| ----------------- |:-----------------:|
| HP_Z2 I915        | 84:a9:3e:71:dd:95 |
| Tektronix MDO3054 | 08:00:11:21:f9:61 |
| DELL_YJCH_Home    | 14:fe:b5:b4:37:9f |
| 550XT2.p0         | b4:96:91:60:40:f0 |
| 550XT2.p1         | b4:96:91:60:40:f1 |
| Thinkpad 1G       | 98:fa:9b:a9:e6:b2 |


In [33]:
ret = sw.pyGSW_MAC_TableEntryRead(1, pr = 0)
print ("ret = ", sw.nRet[ret["ret"]])
for a in range(100):
    print ("nMAC[6] =", ret["nMAC[6]"], ", Pid =", ret["nPortId"], ", nFId =", ret["nFId"], ", nAgeTimer =", ret["nAgeTimer"], )
    if int(ret["bLast"]) == 1:
        break
    ret = sw.pyGSW_MAC_TableEntryRead(0, pr=0)
    

ret =  GSW_statusOk
nMAC[6] = [ 0xb4 0x96 0x91 0x60 0x40 0xf0]  , Pid = 2 , nFId = 0 , nAgeTimer = 300
nMAC[6] = [ 0xb4 0x96 0x91 0x76 0xff 0x35]  , Pid = 1 , nFId = 0 , nAgeTimer = 300


In [10]:
ret = sw.pyGSW_PortCfgGet(0)

GSW_PortCfgGet    ---    Return value: GSW_statusOk 
nPortId = 0x0, 0 
eEnable = 0x1, 1 GSW_PORT_ENABLE_RXTX  [GSW_portEnable_t]
bUnicastUnknownDrop = 0x0, 0 
bMulticastUnknownDrop = 0x0, 0 
bReservedPacketDrop = 0x0, 0 
bBroadcastDrop = 0x0, 0 
bAging = 0x0, 0 
bLearning = 0x0, 0 
bLearningMAC_PortLock = 0x0, 0 
nLearningLimit = 0xff, 255 
bMAC_SpoofingDetection = 0x0, 0 
eFlowCtrl = 0x1, 1 GSW_FLOW_RX  [GSW_portFlow_t]
ePortMonitor = 0x0, 0 GSW_PORT_MONITOR_NONE  [GSW_portMonitor_t]


In [7]:
pce = sw.clear_pce_data()
p = pce[0]
a = pce[1]
p["nIndex"] = 0
p["bEnable"] = 1
p["bPordIdEnable"] = 1
p["nPortId"] = 4
p["bDstIP_Select"] = 1
p["nDstIPipv4"] = 1*0x1000000
print (p["nDstIPipv4"])
p["nDstIP_Mask"] = 0x7e
a["ePortMapAction"] = 4
a["nForwardPortMap"] = 0x40
a["bPortBitMapMuxControl"] = 0
sw.pyGSW_PceRuleWrite(p, a)

16777216


{'ret': 0}

In [13]:
sw.pyGSW_PceRuleRead(1)

param.pattern.nDstIP.nIPv4 = 00000000, 0 
PCE rule index 1, enabled = 1
pattern:
   , bPortIdEnable = 0, nPortId = 0
   , bDSCP_Enable = 0, nDSCP = 0
   , bPCP_Enable = 0, nPCP = 0
   , bSTAG_PCP_DEI_Enable = 0, nSTAG_PCP_DEI = 0
   , bPktLngEnable = 0, nPktLng = 0, nPktLngRange = 0
   , bMAC_DstEnable = 0, nMAC_Dst = 00:00:00:00:00:00, nMAC_DstMask = 0
   , bMAC_SrcEnable = 0, nMAC_Src = 00:00:00:00:00:00, nMAC_SrcMask = 0
   , bAppDataMSB_Enable = 0, nAppDataMSB = 0
   , bAppMaskRangeMSB_Select = 0, nAppMaskRangeMSB = 0
   , bAppDataLSB_Enable = 0, nAppDataLSB = 0
   , bAppMaskRangeLSB_Select = 0, nAppMaskRangeLSB = 0
   , eDstIP_Select = 0, nDstIPipv4 = 0.0.0.0, nDstIPipv6 = 0000:0000:0000:0000, nDstIP_Mask = 0
   , eSrcIP_Select = 0, nSrcIPipv4 = 0.0.0.0, nSrcIPipv6 = 0000:0000:0000:0000, nSrcIP_Mask = 0
   , bEtherTypeEnable = 0, nEtherType = 0, nEtherTypeMask = 0
   , bProtocolEnable = 0, nProtocol = 0, nProtocolMask = 0
   , bSessionIdEnable = 0, nSessionId = 0, bVid = 0, nVid =

{'ret': 0}

#### Reconnect the serial port 
And enable the UART - MDIO bridge

In [16]:
sw.rc()

Now close port at: com24
Open serial port at: com24
('r(0xf383) = ', '0000')
('r(0xf384) = ', '003c')
('w(0xf383) = ', '0003')
('w(0xf384) = ', '003f')


#### Close the serial port

In [43]:
sw.s.close_port()

'Now close port at: com15'

#### Port summary

In [5]:
for a in range(5):
    sw.Phy_sum(a)
    print (" ")

Phy: 0    Link state: 1.0     Speed: 1000 Mbps
Master(1)/Slave(0): 0.0     Local receiver OK: 1.0     Remote receiver OK: 1.0
SNR margin: 8.0     Cable length: 150 m
 
Phy: 1    Link state: 1.0     Speed: 1000 Mbps
Master(1)/Slave(0): 1.0     Local receiver OK: 1.0     Remote receiver OK: 1.0
SNR margin: 8.0     Cable length: 150 m
 
Phy: 2    Link state: 0.0     Speed: 1000 Mbps
Master(1)/Slave(0): 0.0     Local receiver OK: 0.0     Remote receiver OK: 0.0
SNR margin: 15.0     Cable length: 150 m
 
Phy: 3    Link state: 0.0     Speed: 10 Mbps
Master(1)/Slave(0): 0.0     Local receiver OK: 0.0     Remote receiver OK: 0.0
SNR margin: 15.0     Cable length: 150 m
 
Phy: 4    Link state: 1.0     Speed: 10000 Mbps
Master(1)/Slave(0): 1.0     Local receiver OK: 1.0     Remote receiver OK: 1.0
SNR margin: 15.0     Cable length: 150 m
 


#### 2 * EASY GPY211 back to back throughput test
Use 3 ports (port 0, 2, 3) of GWT140 as trunk to connect 2 * EASY GPY211. Configure:

**Board #0:** 

MAC_TableEntryAdd 0 0 1000000 0 1 0 00:22:A2:00:00:01

MAC_TableEntryAdd 0 2 1000000 0 1 0 00:22:A2:00:00:03

MAC_TableEntryAdd 0 3 1000000 0 1 0 00:22:A2:00:00:05

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:00

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:02

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:04

Traffic input from Port 4, the 2.5G PHY, des MAC are 00:22:A2:00:00:01/03/05

**Board #1:** 

MAC_TableEntryAdd 0 0 1000000 0 1 0 00:22:A2:00:00:00

MAC_TableEntryAdd 0 2 1000000 0 1 0 00:22:A2:00:00:02

MAC_TableEntryAdd 0 3 1000000 0 1 0 00:22:A2:00:00:04

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:01

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:03

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:05

Traffic input from Port 4, the 2.5G PHY, des MAC are 00:22:A2:00:00:00/02/04

pyGSW_MAC_TableEntryAdd(nFId, nPortId, nAgeTimer, nSVLAN_Id, bStaticEntry, nTrafficClass, nMAC_0, nMAC_1, nMAC_2, nMAC_3, nMAC_4, nMAC_5):


In [33]:
#Board #0
for a in range(4):
    sw.pyGSW_MAC_TableEntryAdd(0, 0, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x01+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 1, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x03+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 2, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x05+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 3, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x07+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x00+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x02+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x04+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x06+a*8)

In [6]:
#Board #1
for a in range(1):
    sw.pyGSW_MAC_TableEntryAdd(0, 0, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x00+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 1, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x02+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 2, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x04+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 3, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x06+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x01+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x03+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x05+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x07+a*8)